In [ ]:
# REST Inference

## Setup

Change that following variable settings match your deployed model's *Inference endpoint*. for example: 

```
deployed_model_name = "fraud"
infer_endpoint = "https://fraud-predictor-userx-workshop.apps.clusterx.sandboxx.opentlc.com"
```

In [15]:
deployed_model_name = "fraud"
infer_endpoint = "https://fraud-fraud-detection-eder.apps.elyra-ide.osp.rh-ods.com"
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

## Request Function

Build and submit the REST request. 

Note: You submit the data in the same format that you used for an ONNX inference.

In [16]:
import requests


def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "x",
                "shape": [1, 5],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=False) 
    
    # Check the response content for debugging
    print("Response content:", response.content)  # Print the raw response
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

In [17]:
#Load the scaler
import pickle
with open('artifact/scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)

In [18]:
data = [0.3111400080477545, 1.9459399775518593, 1.0, 0.0, 0.0]
prediction = rest_request(scaler.transform([data]).tolist()[0])
prediction

Response content: b'{\n    "model_name": "fraud",\n    "model_version": "1",\n    "outputs": [{\n            "name": "Identity:0",\n            "shape": [1, 1],\n            "datatype": "FP32",\n            "data": [0.0027730322908610107]\n        }]\n}'


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'fraud-fraud-detection-eder.apps.elyra-ide.osp.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[0.0027730322908610106]

In [19]:
threshhold = 0.95

if (prediction[0] > threshhold):
    print('fraud')
else:
    print('not fraud')

not fraud


## Example 1: user buys a coffee

In this example, the user is buying a coffee. The parameters given to the model are:
* same location as the last transaction (distance=0)
* same median price as the last transaction (ratio_to_median=1)
* using a pin number (pin=1)
* using the credit card chip (chip=1)
* not an online transaction (online=0)

In [13]:
data = [0.0, 1.0, 1.0, 1.0, 0.0]
prediction = rest_request(scaler.transform([data]).tolist()[0])
prediction
threshhold = 0.95

if (prediction[0] > threshhold):
    print('The model predicts that this is fraud')
else:
    print('The model predicts that this is not fraud')

Response content: b'{\n    "model_name": "fraud",\n    "model_version": "1",\n    "outputs": [{\n            "name": "Identity:0",\n            "shape": [1, 1],\n            "datatype": "FP32",\n            "data": [2.469630888679575e-10]\n        }]\n}'
The model predicts that this is not fraud


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'fraud-fraud-detection-eder.apps.elyra-ide.osp.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


## Example 2: fraudulent transaction

In this example, someone stole the user's credit card and is buying something online. The parameters given to the model are:
* very far away from the last transaction (distance=100)
* median price similar to the last transaction (ratio_to_median=1.2)
* not using a pin number (pin=0)
* not using the credit card chip (chip=0)
* is an online transaction (online=1)

In [14]:
data = [100, 1.2, 0.0, 0.0, 1.0]
prediction = rest_request(scaler.transform([data]).tolist()[0])
prediction
threshhold = 0.95

if (prediction[0] > threshhold):
    print('The model predicts that this is fraud')
else:
    print('The model predicts that this is not fraud')

Response content: b'{\n    "model_name": "fraud",\n    "model_version": "1",\n    "outputs": [{\n            "name": "Identity:0",\n            "shape": [1, 1],\n            "datatype": "FP32",\n            "data": [0.9953203201293945]\n        }]\n}'
The model predicts that this is fraud


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'fraud-fraud-detection-eder.apps.elyra-ide.osp.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
